[TEMPLATE] — Review and adapt

🧠 Install Semantic Kernel

To use Microsoft’s Semantic Kernel for orchestrating AI agents, planners, and connectors, install the latest version from PyPI using pip.
Run the following command in a code cell below:

In [ ]:
# TEMPLATE: review/adapt this cell for your project
pip install -U semantic-kernel

[TEMPLATE] — Review and adapt

💡 Note: Removing pydrive2

If you see dependency warnings about cryptography or pyOpenSSL when installing Azure packages, they usually come from pydrive2, which Colab preinstalls for legacy Google Drive APIs.
You don’t actually need pydrive2 for Drive access anymore — Colab’s built-in drive.mount() handles everything.

To remove it safely and avoid version conflicts, run:


In [ ]:
# TEMPLATE: review/adapt this cell for your project
!pip -q uninstall -y pydrive2

[TEMPLATE] — Review and adapt

🤖 Connect to Azure OpenAI in Colab

The following code shows how to connect Google Colab to your Azure OpenAI resource using the official openai Python package.

Import the AzureOpenAI client class.

Initialize it with your Azure OpenAI endpoint, API key, and API version.

Use your deployment name (for example, gpt-35-turbo) as the model parameter when sending chat requests.

Print the assistant’s response.

⚠️ Security Note: Never share your real API key in public notebooks or commits.
Instead, store it in an environment variable (for example,
os.environ["AZURE_OPENAI_API_KEY"]) and reference it securely in your code.

Run the next code cell to send a test message to your Azure OpenAI deployment and confirm the connection from Colab.



[TEMPLATE] — Review and adapt

🧠 Installing Microsoft Agentic Framework Dependencies

Before running Azure OpenAI or the Microsoft Agent Framework in Colab, you’ll need to install the latest compatible SDKs and libraries.
This command ensures that Semantic Kernel, Azure AI Agents, and OpenAI are all properly aligned to avoid version conflicts.

This installs:

🧩 semantic-kernel – orchestrates AI workflows and prompt pipelines

🔐 azure-identity – handles secure authentication with Azure services

☁️ azure-ai-projects – project management and AI service integration

🤖 azure-ai-agents – enables creation of tool-using and multi-agent systems

💬 openai – communicates with Azure OpenAI endpoints for completions and chat

✅ Tip: Run this cell once per Colab session before importing any Azure or Semantic Kernel packages.

In [ ]:
# TEMPLATE: review/adapt this cell for your project
!pip -q install -U "semantic-kernel==1.37.1" "azure-identity>=1.25.1" "azure-ai-projects==1.1.0b4" "azure-ai-agents==1.2.0b5" "openai>=1.98"


[TEMPLATE] — Review and adapt

🔑 Configure Azure OpenAI Environment Variables

Set up your Azure credentials so Colab can authenticate directly with your Azure OpenAI resource.
Replace the sample API key and endpoint below with your own values from the Azure Portal (Keys and Endpoint section).

In [ ]:
# TEMPLATE: review/adapt this cell for your project
import os
os.environ["AZURE_OPENAI_ENDPOINT"]   = "AZURE_OPENAI_ENDPOINT"
os.environ["AZURE_OPENAI_API_KEY"]    = "AZURE_OPENAI_API_KEY"   # from Keys & Endpoints
os.environ["AZURE_OPENAI_API_VERSION"]= "2024-10-21"
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "gpt-35-turbo"    # your deployment name


[TEMPLATE] — Review and adapt

✅ Quick sanity check: call your Azure OpenAI deployment

Run this cell to verify your environment variables and deployment are working. It should print a short greeting; if anything’s misconfigured, you’ll see a clear error.

In [ ]:
# TEMPLATE: review/adapt this cell for your project
from openai import AzureOpenAI, OpenAIError
client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
)

try:
    r = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT"],
        messages=[{"role":"user","content":"Say hi in one short line."}]
    )
    print(r.choices[0].message.content)
except OpenAIError as e:
    print("ERROR:", e)

[TEMPLATE] — Review and adapt

🧪 Tool-calling demo: simple agent with a timezone tool

This cell shows a minimal agent loop using Azure OpenAI tool-calling. The model decides when to call the get_timezone tool, you run the tool in Python, then feed the result back for a final answer.

In [ ]:
# TEMPLATE: review/adapt this cell for your project
import json, datetime as dt

def get_timezone(city:str)->str:
    # stub tool; replace with real logic as needed
    return f"Timezone for {city}: US/Eastern (stub) @ {dt.datetime.now().isoformat(timespec='seconds')}"

tools = [{
    "type":"function",
    "function":{
        "name":"get_timezone",
        "description":"Return a timezone for a US city",
        "parameters":{"type":"object",
                      "properties":{"city":{"type":"string"}},
                      "required":["city"]},
    },
}]

messages=[{"role":"system","content":"You are concise and call tools when helpful."},
          {"role":"user","content":"What timezone is Tallahassee in?"}]

resp = client.chat.completions.create(
    model=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    messages=messages,
    tools=tools,
    tool_choice="auto",
)
msg = resp.choices[0].message
messages.append(msg)

if msg.tool_calls:
    for call in msg.tool_calls:
        args = json.loads(call.function.arguments or "{}")
        result = get_timezone(**args)
        messages.append({
            "role":"tool",
            "tool_call_id":call.id,
            "name":call.function.name,
            "content": result
        })
    final = client.chat.completions.create(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT"],
        messages=messages
    )
    print(final.choices[0].message.content)
else:
    print(msg.content)


[TEMPLATE] — Review and adapt

⚙️ Install Microsoft Agent Framework (Preview)

Run this cell to install the Microsoft Agent Framework (pre-release version) into your Colab environment:

In [ ]:
# TEMPLATE: review/adapt this cell for your project
pip install agent-framework --pre